This implementation works fine but it seems too slow for what it does, I switched from einsum to tensordot, that 2x the speed.  

In [30]:
import numpy as np
import plotly.express as px
from numpy.lib.stride_tricks import sliding_window_view

from cifar_10_dataset_loading import load_cifar_10

In [31]:
x_train, y_train, x_test, y_test = load_cifar_10()

In [32]:
IMAGES_IDX = [0, 392, 351, 333]
images = x_train[IMAGES_IDX]
px.imshow(images, facet_col=0)

In [33]:
kernels = np.asarray(
[
    [
        # First kernel
        # First channel
            [
            [1, 0, -1, 0, 1], 
            [1, 0, -1, 0, 1],
            [1, 0, -1, 0, 1],
            [1, 0, -1, 0, 1],
            [1, 0, -1, 0, 1],
        ],
        # Second channel
        [
            [1, 0, 1, 1, 1], 
            [1, 0, 1, 1, 1], 
            [1, 0, 1, 1, 1],
            [1, 0, 1, 1, 1],
            [1, 0, 1, 1, 1],
        ],
        # Third channel
        [
            [0, 0, 0, 0, 1],
            [0, 0, 0, 1, 0],
            [0, 0, 1, 0, 0],
            [0, 1, 0, 0, 0],
            [1, 0, 0, 0, 0],
        ],
    ],
    [
        # Second kernel
        # First channel
        [
            [0.5, 0.5, 0.5, 0.5, 0.5], 
            [0.5, 0.5, 0.5, 0.5, 0.5], 
            [0.5, 0.5, 0.5, 0.5, 0.5], 
            [0.5, 0.5, 0.5, 0.5, 0.5],
            [0.5, 0.5, 0.5, 0.5, 0.5],
        ],
        # Second channel
        [
            [1, 2, 3, 4, 5],
            [1, 1, 1, 0, 0],
            [0, 1, 1, 1, 0],
            [0, 0, 1, 1, 1],
            [0, 1, 0, 1, 1],
        ],
        # Thrid channel
        [
            [0, 0, 1, 0, 0],
            [0, 0.5, 1, 0.5, 0],
            [1, 1, 1, 1, 1],
            [0, 0.5, 1, 0.5, 0],
            [0, 0, 1, 0, 0],
        ],
    ]
]).swapaxes(1, 3)
k.shape

(2, 5, 5, 3)

In [34]:
views = sliding_window_view(images, (k.shape[1], k.shape[1]), (1, 2))
views.shape

(4, 28, 28, 3, 5, 5)

In [35]:
px.imshow(views[0, 5].swapaxes(1, 3), facet_col=0)

In [36]:
views.shape

(4, 28, 28, 3, 5, 5)

In [37]:
k.shape

(2, 5, 5, 3)

In [38]:
correlations = np.einsum("bijcxy, kxyc -> bijk", views, k)

In [40]:
px.imshow(correlations[2], facet_col=2)

In [67]:
def valid_correlate(inputs:np.ndarray, kernels:np.ndarray) -> np.ndarray:
    return np.einsum(
        "bijcxy, kxyc -> bijk",
        sliding_window_view(inputs, (kernels.shape[1], kernels.shape[1]), (1, 2)),
        kernels
    )

def full_convolve(inputs:np.ndarray, k:np.ndarray) -> np.ndarray:
    pad = ((0, 0), (k.shape[1]-1, k.shape[1]-1), (k.shape[2]-1, k.shape[2]-1), (0, 0))
    return valid_correlate(np.pad(inputs, pad, "constant"), np.flip(k, (1, 2)))

convolutions = full_convolve(images, kernels)
px.imshow(convolutions[2], facet_col=2)
